In [ ]:
import numpy as np
import pandas as pd 
import o

In [ ]:

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random
import cv2
from PIL import Image
import seaborn as sns
import time


import torch
import torch.nn as nn
from torchvision import models
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import transforms



In [ ]:

train_dir = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
valid_dir = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
classes = os.listdir(train_dir)
print(f"Classes found: {classes}")
print(f"Number of classes: {len(classes)}")

class_counts = {cls: len(os.listdir(os.path.join(train_dir, cls))) for cls in classes}

In [ ]:
plt.figure(figsize=(12, 8))
plt.barh(list(class_counts.keys()), list(class_counts.values()))
plt.xlabel("Number of Images")
plt.ylabel("Classes")
plt.title("Number of Images per Class in Training Set")
plt.show()

In [ ]:
def display_random_images(data_dir, classes, num_images=5):
    sample_classes = random.sample(classes, num_images)
    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))  
    for i, cls in enumerate(sample_classes):    
        class_path = os.path.join(data_dir, cls)
        img_name = random.choice(os.listdir(class_path))
        img_path = os.path.join(class_path, img_name)

        img = Image.open(img_path)
        axes[i].imshow(img)
        axes[i].set_title(cls)
        axes[i].axis('off')
    
    plt.show()

display_random_images(train_dir, classes)


## Preprocessing Step

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

def apply_filters(image):
    # Convert PIL image to OpenCV format
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Apply Gaussian filtering
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Apply sharpening
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    image = cv2.filter2D(image, -1, kernel)

    # Convert back to PIL format
    image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    return image

# Update transformations with the custom filter function
train_transforms = transforms.Compose([
    transforms.Lambda(apply_filters),                   # Apply custom filters
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_transforms = transforms.Compose([
    transforms.Lambda(apply_filters),                   # Apply custom filters
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:


# Parameters to test
batch_sizes = [32, 64, 128, 256]
num_workers_options = [0, 2, 4, 8]

# Test each combination
results = []
for batch_size in batch_sizes:
    for num_workers in num_workers_options:
        # Define data loader with current settings
        loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers)
        
        # Measure loading time for a small number of batches (e.g., 10)
        start_time = time.time()
        for i, (images, labels) in enumerate(loader):
            if i >= 10:  # Only load 10 batches to test speed
                break
        end_time = time.time()
        
        # Record the result
        time_taken = end_time - start_time
        results.append((batch_size, num_workers, time_taken))
        print(f"Batch size: {batch_size}, Num workers: {num_workers} -> Time: {time_taken:.4f} seconds")

# Find the best configuration
best_config = min(results, key=lambda x: x[2])
print(f"\nOptimal configuration - Batch size: {best_config[0]}, Num workers: {best_config[1]}, Time: {best_config[2]:.4f} seconds")


In [ ]:


# Define the training and validation datasets
train_dataset = ImageFolder(root=train_dir, transform=train_transforms)
valid_dataset = ImageFolder(root=valid_dir, transform=valid_transforms)

# Update data loaders with optimal settings
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False, num_workers=4)


## VGG-16

In [ ]:


# Load pre-trained VGG16 model
model = models.vgg16(pretrained=True)

# Modify the final layer for 38 classes
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, 38)



In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Early stopping parameters
patience = 3
best_val_loss = float('inf')
early_stop_counter = 0

# Lists to store metrics
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    train_loss, correct, total = 0, 0, 0
    
    # Training phase
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()                # Clear gradients
        outputs = model(inputs)              # Forward pass
        loss = criterion(outputs, labels)    # Compute loss
        loss.backward()                      # Backpropagation
        optimizer.step()                     # Update weights

        # Calculate training accuracy
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_accuracy = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
    
    # Validation phase
    model.eval()  # Set model to evaluation mode
    valid_loss, correct, total = 0, 0, 0
    with torch.no_grad():  # No gradient tracking
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            
            # Calculate validation accuracy
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    valid_accuracy = 100 * correct / total
    val_losses.append(valid_loss)
    val_accuracies.append(valid_accuracy)
    print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%\n")
    
    # Early stopping logic
    if valid_loss < best_val_loss:
        best_val_loss = valid_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

# Plot training and validation metrics
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()


In [ ]:
# Save the model state dictionary
torch.save(model.state_dict(), 'plant_disease_classification_model_vgg.pth')

In [ ]:
# Function to get predictions and labels for a model
def get_predictions_labels(model, data_loader):
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return all_preds, all_labels

In [ ]:
from sklearn.metrics import classification_report
import torch

# Load the saved models
# VGG16
model_vgg16 = models.vgg16(pretrained=False)
model_vgg16.classifier[6] = nn.Linear(model_vgg16.classifier[6].in_features, 38)
model_vgg16.load_state_dict(torch.load('plant_disease_classification_model_vgg.pth'))
model_vgg16 = model_vgg16.to(device)
model_vgg16.eval()
# Get predictions and labels for both models
vgg16_preds, vgg16_labels = get_predictions_labels(model_vgg16, valid_loader)

# Generate and print classification reports
print("Classification Report for VGG16 Model:\n")
print(classification_report(vgg16_labels, vgg16_preds))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(true_labels, predicted_labels, model_name, class_names):
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    
    plt.figure(figsize=(16, 14))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="inferno", cbar=True, xticklabels=class_names, yticklabels=class_names)
    plt.title(f"Confusion Matrix for {model_name} Model")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.show()



In [ ]:
# Class names (replace with actual class names if you have them)
class_names = [f"Class {i}" for i in range(38)]  # Replace with actual class names if available

# Plot confusion matrix for VGG16
plot_confusion_matrix(vgg16_labels, vgg16_preds, "VGG16", class_names)

## ResNet-50

In [ ]:
# Load the pre-trained ResNet50 model
model_resnet = models.resnet50(pretrained=True)

# Modify the final layer for 38 classes
num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, 38)

# Move the model to GPU if available
model_resnet = model_resnet.to(device)


In [ ]:
# Define loss function and optimizer for ResNet50
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet.parameters(), lr=0.001)


# Early stopping parameters
patience = 3
best_val_loss = float('inf')
early_stop_counter = 0

# Lists to store metrics
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model_resnet.train()  # Set model to training mode
    train_loss, correct, total = 0, 0, 0
    
    # Training phase
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()                # Clear gradients
        outputs = model_resnet(inputs)        # Forward pass
        loss = criterion(outputs, labels)    # Compute loss
        loss.backward()                      # Backpropagation
        optimizer.step()                     # Update weights

        # Calculate training accuracy
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_accuracy = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
    
    # Validation phase
    model_resnet.eval()  # Set model to evaluation mode
    valid_loss, correct, total = 0, 0, 0
    with torch.no_grad():  # No gradient tracking
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_resnet(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            
            # Calculate validation accuracy
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    valid_accuracy = 100 * correct / total
    val_losses.append(valid_loss)
    val_accuracies.append(valid_accuracy)
    print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%\n")
    
    # Early stopping logic
    if valid_loss < best_val_loss:
        best_val_loss = valid_loss
        early_stop_counter = 0
        torch.save(model_resnet.state_dict(), 'best_resnet50_model.pth')  # Save the best model
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

In [ ]:
# Plot training and validation metrics
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

In [ ]:
# Save the model state dictionary
torch.save(model_resnet.state_dict(), 'plant_disease_classification_model_resnet50.pth')

In [ ]:
from sklearn.metrics import classification_report
import torch

# ResNet50
model_resnet50 = models.resnet50(pretrained=False)
model_resnet50.fc = nn.Linear(model_resnet50.fc.in_features, 38)
model_resnet50.load_state_dict(torch.load('plant_disease_classification_model_resnet50.pth'))
model_resnet50 = model_resnet50.to(device)
model_resnet50.eval()

resnet50_preds, resnet50_labels = get_predictions_labels(model_resnet50, valid_loader)

print("Classification Report for ResNet50 Model:\n")
print(classification_report(resnet50_labels, resnet50_preds))


In [ ]:
# Class names (replace with actual class names if you have them)
class_names = [f"Class {i}" for i in range(38)]  # Replace with actual class names if available
# Plot confusion matrix for ResNet50
plot_confusion_matrix(resnet50_labels, resnet50_preds, "ResNet50", class_names)

## Inception-V3

In [ ]:
# Define preprocessing for InceptionV3
inception_transforms = transforms.Compose([
    transforms.Resize((299, 299)),                  # Resize to 299x299
    transforms.RandomHorizontalFlip(),              # Apply other augmentations as needed
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Make sure to apply these transformations in your DataLoader for InceptionV3
train_dataset = ImageFolder(root=train_dir, transform=inception_transforms)
valid_dataset = ImageFolder(root=valid_dir, transform=inception_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)


In [ ]:
# Load InceptionV3 model
model_inception = models.inception_v3(pretrained=True)
model_inception.aux_logits = False  # Disable auxiliary logits

# Modify the final layer
num_features = model_inception.fc.in_features
model_inception.fc = nn.Linear(num_features, 38)

# Move model to GPU if available
model_inception = model_inception.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_inception.parameters(), lr=0.001)

In [ ]:
#Early stopping parameters
patience = 3
best_val_loss = float('inf')
early_stop_counter = 0

# Lists to store metrics
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model_inception.train()  # Set model to training mode
    train_loss, correct, total = 0, 0, 0
    
    # Training phase
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()                # Clear gradients
        outputs = model_inception(inputs)        # Forward pass
        loss = criterion(outputs, labels)    # Compute loss
        loss.backward()                      # Backpropagation
        optimizer.step()                     # Update weights

        # Calculate training accuracy
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    train_accuracy = 100 * correct / total
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
    
    # Validation phase
    model_inception.eval()  # Set model to evaluation mode
    valid_loss, correct, total = 0, 0, 0
    with torch.no_grad():  # No gradient tracking
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model_inception(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()
            
            # Calculate validation accuracy
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    valid_accuracy = 100 * correct / total
    val_losses.append(valid_loss)
    val_accuracies.append(valid_accuracy)
    print(f"Validation Loss: {valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.2f}%\n")
    
    # Early stopping logic
    if valid_loss < best_val_loss:
        best_val_loss = valid_loss
        early_stop_counter = 0
        torch.save(model_inception.state_dict(), 'best_inceptionV3_model.pth')  # Save the best model
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print("Early stopping triggered")
            break

In [ ]:
# Plot training and validation metrics
plt.figure(figsize=(12, 5))

# Plot loss
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

# Plot accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.show()

## Evaluation

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

# Load the Inception v3 model with pre-trained weights
model_inception = models.inception_v3(weights=models.Inception_V3_Weights.IMAGENET1K_V1)

# Modify the final fully connected layer to match the number of classes in your dataset
num_classes = 38  # Replace with the actual number of classes
model_inception.fc = nn.Linear(model_inception.fc.in_features, num_classes)

# Load your trained model's state dictionary
model_inception.load_state_dict(torch.load('best_inceptionV3_model.pth'))

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_inception = model_inception.to(device)
model_inception.eval()  # Set the model to evaluation mode

# Get predictions and labels for InceptionV3
inception_preds, inception_labels = get_predictions_labels(model_inception, valid_loader)

# Print classification report
print("Classification Report for InceptionV3 Model:\n")
print(classification_report(inception_labels, inception_preds))

# Class names (replace with actual class names if you have them)
class_names = [f"Class {i}" for i in range(38)]  # Replace with actual class names if available

# Plot confusion matrix for InceptionV3
plot_confusion_matrix(inception_labels, inception_preds, "InceptionV3", class_names)

# Evaluation on Test Dataset

In [ ]:
class_names = train_dataset.classes  # This will give you the class names in the order used during training
print(class_names)

In [ ]:
import os
import torch
from PIL import Image
from sklearn.metrics import classification_report
from torchvision import transforms, models
import torch.nn as nn

# Define the correct class names in the order used during training
class_names = [
    'Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust', 'Apple___healthy',
    'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy',
    'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot', 'Corn_(maize)___Common_rust_',
    'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy', 'Grape___Black_rot',
    'Grape___Esca_(Black_Measles)', 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 'Grape___healthy',
    'Orange___Haunglongbing_(Citrus_greening)', 'Peach___Bacterial_spot', 'Peach___healthy',
    'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', 'Potato___Early_blight',
    'Potato___Late_blight', 'Potato___healthy', 'Raspberry___healthy', 'Soybean___healthy',
    'Squash___Powdery_mildew', 'Strawberry___Leaf_scorch', 'Strawberry___healthy',
    'Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___Late_blight',
    'Tomato___Leaf_Mold', 'Tomato___Septoria_leaf_spot', 'Tomato___Spider_mites Two-spotted_spider_mite',
    'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy'
]

# Define the mapping from simple class names to full class names in class_names
simple_to_full_class_mapping = {
    'TomatoEarlyBlight': 'Tomato___Early_blight',
    'TomatoYellowCurlVirus': 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    'PotatoHealthy': 'Potato___healthy',
    'PotatoEarlyBlight': 'Potato___Early_blight',
    'CornCommonRust': 'Corn_(maize)___Common_rust_',
    'AppleScab': 'Apple___Apple_scab',
    'TomatoHealthy': 'Tomato___healthy',
    'AppleCedarRust': 'Apple___Cedar_apple_rust'
}

# Define the test directory path
test_dir = '/kaggle/input/new-plant-diseases-dataset/test/test'

# Define the same preprocessing transformations used during training
test_transforms = transforms.Compose([
    transforms.Resize((299, 299)),                  # Resize to 299x299 for InceptionV3
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Standard normalization for ImageNet
])

# Load the pre-trained model with the correct output layer
model_inception = models.inception_v3(pretrained=False)
model_inception.fc = nn.Linear(model_inception.fc.in_features, len(class_names))  # Set the output layer size
model_inception.load_state_dict(torch.load('best_inceptionV3_model.pth'))
model_inception = model_inception.to(device)
model_inception.eval()  # Set the model to evaluation mode

# Lists to store true and predicted labels
true_labels = []
predicted_labels = []

# Loop through each image in the test directory
for image_file in os.listdir(test_dir):
    # Extract the base class name (e.g., 'TomatoEarlyBlight' from 'TomatoEarlyBlight6.JPG')
    base_class_name = ''.join([i for i in image_file if not i.isdigit()]).replace(".JPG", "")
    
    # Map the simplified name to the full name in `class_names`
    true_class_name = simple_to_full_class_mapping.get(base_class_name, None)
    
    # Find the index in `class_names` if mapping is successful
    if true_class_name:
        true_label = class_names.index(true_class_name)
    else:
        print(f"Warning: {base_class_name} not found in class names.")
        continue  # Skip if class not found in mapping
    
    # Load and preprocess the image
    image_path = os.path.join(test_dir, image_file)
    image = Image.open(image_path).convert("RGB")  # Ensure the image is in RGB format
    image = test_transforms(image).unsqueeze(0).to(device)
    
    # Get model prediction
    with torch.no_grad():
        output = model_inception(image)
        _, predicted_label = torch.max(output, 1)
    
    # Append to results
    true_labels.append(true_label)
    predicted_labels.append(predicted_label.item())
    
    # Print individual predictions (optional)
    print(f"Image: {image_file} | True Label: {true_label} ({true_class_name}) | Predicted Label: {predicted_label.item()} ({class_names[predicted_label.item()]})")




In [ ]:
from sklearn.metrics import classification_report

# Find unique labels in true_labels and filter class_names accordingly
unique_labels = sorted(set(true_labels))  # Get sorted list of unique labels in the test set
filtered_class_names = [class_names[i] for i in unique_labels]  # Filter class names to match unique labels

# Generate and print the classification report
print("\nClassification Report for Test Set:\n")
print(classification_report(true_labels, predicted_labels, labels=unique_labels, target_names=filtered_class_names))


## Custom CNN architecture

In [ ]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(128 * 37 * 37, 512)  # Adjust based on input size
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 37 * 37)  # Flatten
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Set up the custom model
num_classes = len(class_names)  # Adjust this based on the number of classes
model_custom = CustomCNN(num_classes).to(device)

In [ ]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 10

# Optimizer and Loss Function
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_custom.parameters(), lr=learning_rate)


In [ ]:
for epoch in range(num_epochs):
    model_custom.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model_custom(inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Print stats
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")


## Evaluation on test set

In [ ]:
# Evaluate on test set
model_custom.eval()
true_labels = []
predicted_labels = []

for image_file in os.listdir(test_dir):
    # Extract the base class name (e.g., 'TomatoEarlyBlight' from 'TomatoEarlyBlight6.JPG')
    base_class_name = ''.join([i for i in image_file if not i.isdigit()]).replace(".JPG", "")
    
    # Map the simplified name to the full name in `class_names`
    true_class_name = simple_to_full_class_mapping.get(base_class_name, None)
    
    # Find the index in `class_names` if mapping is successful
    if true_class_name:
        true_label = class_names.index(true_class_name)
    else:
        print(f"Warning: {base_class_name} not found in class names.")
        continue  # Skip if class not found in mapping
    
    # Load and preprocess the image
    image_path = os.path.join(test_dir, image_file)
    image = Image.open(image_path).convert("RGB")  # Ensure the image is in RGB format
    image = test_transforms(image).unsqueeze(0).to(device)
    
    # Get model prediction
    with torch.no_grad():
        output = model_custom(image)
        _, predicted_label = torch.max(output, 1)
    
    # Append to results
    true_labels.append(true_label)
    predicted_labels.append(predicted_label.item())
    
    # Print individual predictions (optional)
    print(f"Image: {image_file} | True Label: {true_label} ({true_class_name}) | Predicted Label: {predicted_label.item()} ({class_names[predicted_label.item()]})")

# Generate and print the classification report
print("Classification Report for Custom CNN Model on Test Set:\n")
print(classification_report(true_labels, predicted_labels, target_names=class_names))
